In [ ]:
import json
import uproot
import awkward as ak
import vector
import numpy as np
import matplotlib.pyplot as plt
import warnings

# plotting params
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'axes.grid': True,
    'grid.alpha': 0.6,
    'grid.linestyle': '--',
    'font.size': 14,
    "figure.dpi": 200,
})

# Suppress a harmless warning from the vector library with awkward arrays
warnings.filterwarnings("ignore", message="Passing an awkward array to a ufunc")

# Register the vector library with awkward array
ak.behavior.update(vector.backends.awkward.behavior)

# --- CONFIGURATION ---
with open("hh-bbbb-obj-config.json", "r") as config_file:
    CONFIG = json.load(config_file)


In [ ]:
from data_loading_helpers import load_and_prepare_data

events = load_and_prepare_data(
    "/vols/cms/at3722/root-obj-perf/data/hh4b_puppi_pf/hh4b/data_1.root",
    "Events",
    [
        "L1puppiJetSC4NG",      # Jets
        "L1BarrelExtPuppi",      # Candidates
        "GenPart"                # Labels
    ],
    max_events=None
)
l1_col = "L1puppiJetSC4NG"
l1_puppi_col = "L1BarrelExtPuppi"
for idx in range(3):
    print(f"\nEvent {idx}:")
    l1_jets = events[l1_col][idx]
    l1_cands = events[l1_puppi_col][idx]
    print(f"  L1 Jets: {len(l1_jets)}")
    print(f"  L1 Candidates: {len(l1_cands)}")

    plt.scatter(
        l1_cands.phi,
        l1_cands.eta,
        s=5,
        c='gray',
        alpha=0.5,
        label='L1 Candidates'
    )
    plt.scatter(
        l1_jets.phi,
        l1_jets.eta,
        s=50,
        c='red',
        marker='x',
        label='L1 Jets'
    )
    plt.xlabel('Phi')
    plt.ylabel('Eta')
    plt.title(f'L1 Jets and Candidates - Event {idx}')
    plt.legend()
    plt.show()

    l1_jets_expanded = l1_jets
    
    for jdx in range(min(5, len(l1_jets))):
        jet = l1_jets[jdx]
        print(f"    Jet {jdx}: pt={jet.pt:.2f}, eta={jet.eta:.2f}, phi={jet.phi:.2f}")
        # Find constituents within DeltaR < 0.4
        jet_vec = vector.obj(pt=jet.pt, eta=jet.eta, phi=jet.phi)
        constituents_in_jet = []
        for cdx in range(len(l1_cands)):
            cand = l1_cands[cdx]
            cand_vec = vector.obj(pt=cand.pt, eta=cand.eta, phi=cand.phi)
            delta_r = jet_vec.deltaR(cand_vec)
            if delta_r < 0.4:
                constituents_in_jet.append(cand)
        print(f"      Constituents in DeltaR<0.4: {len(constituents_in_jet)}")